In [1]:
with open('input') as f:
    parts = f.read().strip().split()
    boss_stats = {
        'hit_points': int(parts[2]),
        'damage': int(parts[4]),
        'armor': int(parts[6]),
    }

In [2]:
weapons = {
    'Dagger': (8, 4),
    'Shortsword': (10, 5),
    'Warhammer': (25, 6),
    'Longsword': (40, 7),
    'Greataxe': (74, 8),
}

armor = {
    'Leather': (13, 1),
    'Chainmail': (31, 2),
    'Splintmail': (53, 3),
    'Bandedmail': (75, 4),
    'Platemail': (102, 5),
}

rings = {
    'Damage +1': (25, 1, 0),
    'Damage +2': (50, 2, 0),
    'Damage +3': (100, 3, 0),
    'Defense +1': (20, 0, 1),
    'Defense +2': (40, 0, 2),
    'Defense +3': (80, 0, 3),
}

In [3]:
class Player:
    def __init__(self, hit_points, damage, armor):
        self.hit_points = hit_points
        self.damage = damage
        self.armor = armor
        
    def attack(self, other):
        damage = max(self.damage - other.armor, 1)
        other.hit_points -= damage

In [4]:
from collections import deque
from itertools import islice

def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) -> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = deque(islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)

In [5]:
from itertools import cycle

def play_game(me_stats, boss_stats):
    me = Player(**me_stats)
    boss = Player(**boss_stats)
    players = cycle([me, boss])
    for attacker, defender in sliding_window(players, 2):
        attacker.attack(defender)
        if defender.hit_points < 1:
            return attacker == me

In [6]:
def get_weapon_combos():
    yield from weapons

In [7]:
def get_armour_combos():
    yield None
    yield from armor

In [8]:
from itertools import product, combinations

def get_ring_combos():
    yield (None, None)
    yield from ((ring, None) for ring in rings)
    yield from combinations(rings, 2)

In [9]:
def get_combos():
    weapon_combos = get_weapon_combos()
    armor_combos = get_armour_combos()
    ring_combos = get_ring_combos()
    yield from product(weapon_combos, armor_combos, ring_combos)

In [10]:
wins = set()
losses = set()
for chosen_weapon, chosen_armor, chosen_rings in get_combos():
    weapon_cost, weapon_damage = weapons[chosen_weapon]
    armor_cost, armor_value = armor.get(chosen_armor, (0, 0))
    ring1_cost, ring1_damage, ring1_armor = rings.get(chosen_rings[0], (0, 0, 0))
    ring2_cost, ring2_damage, ring2_armor = rings.get(chosen_rings[1], (0, 0, 0))
    
    player_stats = {
        'hit_points': 100,
        'damage': weapon_damage + ring1_damage + ring2_damage,
        'armor': armor_value + ring1_armor + ring2_armor,
    }
    cost = weapon_cost + armor_cost + ring1_cost + ring2_cost
    if play_game(player_stats, boss_stats):
        wins.add(cost)
    else:
        losses.add(cost)

In [11]:
print("Part 1:")
print(min(wins))

Part 1:
121


In [12]:
print("Part 2:")
print(max(losses))

Part 2:
201
